In [1]:
%%file gridlookup2.m
function ix = gridlookup2(x0,xgrid)

nx = size(xgrid,1);

ix = 0;
for jx = 1:nx
    if (x0<=xgrid(jx)); break; end;
    ix = ix+1;
end

ix = min(max(1,ix),nx-1);

Created file '/home/takeki/Dropbox/advmacro2020/aiyagari/gridlookup2.m'.


In [2]:
%%file driver.m
function [r1,vmat0,mu0] = driver(r0,vmat0,mu0,Ge,Pe,knotsb,BETA,ALPHA,DELTA,SIGMA,znow,lnow,critin,critmu)

ne = size(Ge,1);
nb = size(knotsb,1);

vmat1 = zeros(nb,ne);
gmat0 = zeros(nb,ne);

% Factor Price
m0 = lnow*(r0/ALPHA/znow)^(1/(ALPHA-1));
w0 = (1-ALPHA)*znow*m0^(ALPHA)*lnow^(-ALPHA);

% instanteneous utility
umat0 = zeros(nb,nb,ne);
for ie = 1:ne
    for ib = 1:nb
        for jb = 1:nb

            enow = Ge(ie);
            know = knotsb(ib);
            kp = knotsb(jb);
            cnow = w0*enow + (1.0+r0-DELTA)*know - kp;

            if (cnow>0)
                umat0(jb,ib,ie) = log(cnow);
            else
                umat0(jb,ib,ie) = -inf;
            end

        end
    end
end

% value function iteration
diff = 1e+4;
iterin = 0;

while (diff>critin)

    for ie = 1:ne

        utemp = umat0(:,:,ie);
        vcond = Pe(ie,1)*vmat0(:,1) + Pe(ie,2)*vmat0(:,2);
        vtemp = utemp + BETA*vcond;
        [vmat1(:,ie) ivec] = max(vtemp);

%     for ie = 1:ne
%         
%         for ib = 1:nb
% 
%             utemp = umat0(:,ib,ie);
%             vcond = Pe(ie,1)*vmat0(:,1) + Pe(ie,2)*vmat0(:,2);
%             vtemp = utemp + BETA*vcond;
%             vmat1(ib,ie) = max(vtemp);
%             
%         end

    end

    diff = max(max(abs(vmat1-vmat0)));
    iterin = iterin+1;
    vmat0 = vmat1;
    % disp([iterin diff])

end

% policy function
for ie = 1:ne

    for ib = 1:nb

        utemp = umat0(:,ib,ie);
        vcond = Pe(ie,1)*vmat0(:,1) + Pe(ie,2)*vmat0(:,2);
        vtemp = utemp + BETA*vcond;
        [~,jb] = max(vtemp);
        gmat0(ib,ie) = knotsb(jb);

    end

end

% transition matrix
%AA = sparse(nb*ne,nb*ne);
AA = zeros(nb*ne,nb*ne);
wb = zeros(nb,ne);

for ie = 1:ne

    for ib = 1:nb

        know = knotsb(ib);
        kp = gmat0(ib,ie);
        kb = gridlookup2(kp,knotsb);
        wb(ib,ie) = (knotsb(kb+1)-kp)/(knotsb(kb+1)-knotsb(kb));

        for je = 1:ne

            ia = nb*(ie-1)+ib;
            ja = nb*(je-1)+kb;
            AA(ia,ja)   = wb(ib,ie)*Pe(ie,je);
            AA(ia,ja+1) = (1.0-wb(ib,ie))*Pe(ie,je);

        end

    end

end

% distribution
diffmu = 1e+4;
dist0 = reshape(mu0,2*nb,1); 

while (diffmu>critmu)

    dist1 = AA'*dist0;
    diffmu = max(abs(dist1-dist0));
    dist0 = dist1/sum(dist1);

end

mu0 = reshape(dist0,nb,2);

% Calculate K
m1 = 0.0;
for ie = 1:ne

    m1 = m1 + mu0(:,ie)'*knotsb;

end

r1 = (ALPHA)*znow*m1^(ALPHA-1)*lnow^(1-ALPHA);

end


Created file '/home/takeki/Dropbox/advmacro2020/aiyagari/driver.m'.


In [3]:
% Bewley-Aiyagari model
% May 2020, Takeki Sunakawa
clear all;

BETA  = 0.96;
ALPHA = 0.36; 
DELTA = 1-0.92;
SIGMA = 1.5;
mu = 0.05;
pee = 0.925;
puu = 0.5;

% grid points
ne = 2;
Ge = [1.0; mu]; 
Pe = [pee 1-pee;
    1-puu puu]; 

nb = 1001;
kmin = 0;
kmax = 20.0;
knotsb = linspace(kmin,kmax,nb)';
% knotsb = logspace(log(kbounds(1) - 1.0d0*kbounds(1) + 1.0d0)/log(10.0d0), log(kbounds(2) - 1.0d0*kbounds(1) + 1.0d0)/log(10.0d0), nb)';
% knotsb = knotsb + kbounds(1) - 1.0d0;

mue = Pe^10000;
mue = mue(1,:)';
znow = 1.0d0;
% efficiency unit of labor
lnow = Ge'*mue;

% initial distribution
vmat0 = zeros(nb,ne);
mu0 = ones(nb,ne)/(nb*ne);

% initial value of r
mnow = lnow*((1.0-BETA*(1.0-DELTA))/(ALPHA*BETA))^(1.0/(ALPHA-1.0));
mnow = 5.2074; % from My_Aiyagari.m
m0 = mnow;
r0 = (ALPHA)*znow*mnow^(ALPHA-1)*lnow^(1-ALPHA);


start = tic;
critin = 1e-4;
critmu = 1e-8;
critout = 1e-3;
diffout = 1e+3;
damp = 0.01;
iter = 0;
maxiter = 1000;

while (diffout>critout && iter<maxiter)

    [r1,vmat0,mu0] = driver(r0,vmat0,mu0,Ge,Pe,knotsb,BETA,ALPHA,DELTA,SIGMA,znow,lnow,critin,critmu);

    diffout = abs(log(r1)-log(r0));    
    iter = iter+1;    
    disp(sprintf("iter = %4d, diff = %5.6f, oldr = %5.6f, newr = %5.6f",iter,diffout,r0,r1))

    % Update K
    r0 = damp*r1 + (1.0-damp)*r0;
    
end

t = toc(start);
disp(sprintf("Elapsed time is %5.10f.",t));

iter =    1, diff = 0.191966, oldr = 0.115052, newr = 0.139401
iter =    2, diff = 0.179063, oldr = 0.115296, newr = 0.137905
iter =    3, diff = 0.166392, oldr = 0.115522, newr = 0.136435
iter =    4, diff = 0.153681, oldr = 0.115731, newr = 0.134956
iter =    5, diff = 0.142776, oldr = 0.115923, newr = 0.133714
iter =    6, diff = 0.131264, oldr = 0.116101, newr = 0.132387
iter =    7, diff = 0.121640, oldr = 0.116264, newr = 0.131303
iter =    8, diff = 0.111574, oldr = 0.116414, newr = 0.130156
iter =    9, diff = 0.102348, oldr = 0.116552, newr = 0.129112
iter =   10, diff = 0.093758, oldr = 0.116677, newr = 0.128146
iter =   11, diff = 0.086373, oldr = 0.116792, newr = 0.127328
iter =   12, diff = 0.078227, oldr = 0.116897, newr = 0.126409
iter =   13, diff = 0.071472, oldr = 0.116993, newr = 0.125660
iter =   14, diff = 0.064705, oldr = 0.117079, newr = 0.124905
iter =   15, diff = 0.058778, oldr = 0.117158, newr = 0.124250
iter =   16, diff = 0.053440, oldr = 0.117228, newr = 0